<a href="https://colab.research.google.com/github/dlkhang/ViT5-Question-Generation/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#1. Train 5 epoch

vit5

In [ ]:
!pip install -q transformers datasets sentencepiece accelerate

import os, shutil, gc, torch
from google.colab import drive
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# 1. Cấu hình
if not os.path.exists('/content/drive'): drive.mount('/content/drive')
DATA_DIR = "/content/drive/MyDrive/KhoaLuan/data"
TRAIN_FILE = os.path.join(DATA_DIR, "train_grouped_CLEAN_OPTIMIZED.json")
VAL_FILE   = os.path.join(DATA_DIR, "val_grouped_CLEAN_OPTIMIZED.json")
FINAL_DIR  = "/content/drive/MyDrive/KhoaLuan/model_vit5_final_5ep"
TEMP_DIR   = "/content/vit5_temp"

# 2. Chuẩn bị Data
tokenizer = AutoTokenizer.from_pretrained("vietai/vit5-base")
def preprocess(examples):
    inputs = tokenizer(examples["input_text"], max_length=320, padding="max_length", truncation=True)
    targets = tokenizer(text_target=examples["output_text"], max_length=256, padding="max_length", truncation=True)
    targets["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in targets["input_ids"]]
    inputs["labels"] = targets["input_ids"]
    return inputs

dataset = load_dataset("json", data_files={"train": TRAIN_FILE, "validation": VAL_FILE})
tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

# 3. Train
gc.collect(); torch.cuda.empty_cache()
model = AutoModelForSeq2SeqLM.from_pretrained("vietai/vit5-base")

args = Seq2SeqTrainingArguments(
    output_dir=TEMP_DIR,
    learning_rate=3e-4,
    num_train_epochs=5,
    lr_scheduler_type="cosine", warmup_ratio=0.05, label_smoothing_factor=0.1, weight_decay=0.02,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    fp16=True,
    save_strategy="no",
    logging_steps=100, report_to="none"
)

trainer = Seq2SeqTrainer(model=model, args=args, train_dataset=tokenized["train"], eval_dataset=tokenized["validation"], tokenizer=tokenizer, data_collator=DataCollatorForSeq2Seq(tokenizer, model=model))

print("\n🔥 TRAIN ViT5...")
trainer.train()

# 4. Lưu Model
print(f"\n📦 Đang lưu về: {FINAL_DIR}")
if os.path.exists(FINAL_DIR): shutil.rmtree(FINAL_DIR)
trainer.save_model(FINAL_DIR)
tokenizer.save_pretrained(FINAL_DIR)
print("✅ Đã lưu ViT5!")

bart-pho

In [ ]:
!pip install -q transformers datasets sentencepiece accelerate

import os, shutil, gc, torch
from google.colab import drive
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# 1. Cấu hình
if not os.path.exists('/content/drive'): drive.mount('/content/drive')
DATA_DIR = "/content/drive/MyDrive/KhoaLuan/data"
TRAIN_FILE = os.path.join(DATA_DIR, "train_grouped_CLEAN_OPTIMIZED.json")
VAL_FILE   = os.path.join(DATA_DIR, "val_grouped_CLEAN_OPTIMIZED.json")
FINAL_DIR  = "/content/drive/MyDrive/KhoaLuan/model_bartpho_final_5ep"
TEMP_DIR   = "/content/bartpho_temp"

# 2. Chuẩn bị Data
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
def preprocess(examples):
    inputs = tokenizer(examples["input_text"], max_length=320, padding="max_length", truncation=True)
    targets = tokenizer(text_target=examples["output_text"], max_length=256, padding="max_length", truncation=True)
    targets["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in targets["input_ids"]]
    inputs["labels"] = targets["input_ids"]
    return inputs

dataset = load_dataset("json", data_files={"train": TRAIN_FILE, "validation": VAL_FILE})
tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

# 3. Train
gc.collect(); torch.cuda.empty_cache()
model = AutoModelForSeq2SeqLM.from_pretrained("vinai/bartpho-syllable")

args = Seq2SeqTrainingArguments(
    output_dir=TEMP_DIR,
    learning_rate=2e-5,
    num_train_epochs=5,
    lr_scheduler_type="cosine", warmup_ratio=0.05, label_smoothing_factor=0.1, weight_decay=0.02,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    fp16=True,
    save_strategy="no",
    logging_steps=100, report_to="none"
)

trainer = Seq2SeqTrainer(model=model, args=args, train_dataset=tokenized["train"], eval_dataset=tokenized["validation"], tokenizer=tokenizer, data_collator=DataCollatorForSeq2Seq(tokenizer, model=model))

print("\n🔥 TRAIN BARTpho...")
trainer.train()

# 4. Lưu Model
print(f"\n📦 Đang lưu về: {FINAL_DIR}")
if os.path.exists(FINAL_DIR): shutil.rmtree(FINAL_DIR)
trainer.save_model(FINAL_DIR)
tokenizer.save_pretrained(FINAL_DIR)
print("✅ Đã lưu BARTpho!")

t5

In [ ]:
!pip install -q transformers datasets sentencepiece accelerate

import os, shutil, gc, torch
from google.colab import drive
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# 1. Cấu hình
if not os.path.exists('/content/drive'): drive.mount('/content/drive')
DATA_DIR = "/content/drive/MyDrive/KhoaLuan/data"
TRAIN_FILE = os.path.join(DATA_DIR, "train_grouped_CLEAN_OPTIMIZED.json")
VAL_FILE   = os.path.join(DATA_DIR, "val_grouped_CLEAN_OPTIMIZED.json")
FINAL_DIR  = "/content/drive/MyDrive/KhoaLuan/model_t5_base_final_5ep"
TEMP_DIR   = "/content/t5_base_temp"

# 2. Chuẩn bị Data
tokenizer = AutoTokenizer.from_pretrained("t5-base")
def preprocess(examples):
    inputs = tokenizer(examples["input_text"], max_length=320, padding="max_length", truncation=True)
    targets = tokenizer(text_target=examples["output_text"], max_length=256, padding="max_length", truncation=True)
    targets["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in targets["input_ids"]]
    inputs["labels"] = targets["input_ids"]
    return inputs

dataset = load_dataset("json", data_files={"train": TRAIN_FILE, "validation": VAL_FILE})
tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

# 3. Train
gc.collect(); torch.cuda.empty_cache()
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

args = Seq2SeqTrainingArguments(
    output_dir=TEMP_DIR,
    learning_rate=3e-4,
    num_train_epochs=5,
    lr_scheduler_type="cosine", warmup_ratio=0.05, label_smoothing_factor=0.1, weight_decay=0.02,

    # Cấu hình chống lỗi NaN
    fp16=False,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,

    save_strategy="no",
    logging_steps=100, report_to="none"
)

trainer = Seq2SeqTrainer(model=model, args=args, train_dataset=tokenized["train"], eval_dataset=tokenized["validation"], tokenizer=tokenizer, data_collator=DataCollatorForSeq2Seq(tokenizer, model=model))

print("\n🔥 TRAIN t5-base...")
trainer.train()

# 4. Lưu Model
print(f"\n📦 Đang lưu về: {FINAL_DIR}")
if os.path.exists(FINAL_DIR): shutil.rmtree(FINAL_DIR)
trainer.save_model(FINAL_DIR)
tokenizer.save_pretrained(FINAL_DIR)
print("✅ Đã lưu t5-base!")

#2. So sánh 3 mô hình

In [ ]:
# ==============================================================================
# 📊 CODE ĐÁNH GIÁ & SO SÁNH 3 MÔ HÌNH (5 EPOCHS)
# ==============================================================================

# 1. CÀI ĐẶT THƯ VIỆN CHẤM ĐIỂM
print("⏳ Đang cài đặt thư viện evaluate, rouge_score...")
!pip install -q evaluate rouge_score absl-py nltk matplotlib pandas
import nltk
nltk.download('wordnet')
nltk.download('punkt')
print("✅ Cài đặt xong!")

import torch
import json
import gc
import os
import pandas as pd
import matplotlib.pyplot as plt
import evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
from google.colab import drive

# 2. CẤU HÌNH ĐƯỜNG DẪN (ĐÚNG THEO CÁC MODEL 5 EP BẠN VỪA TRAIN)
# ------------------------------------------------------------------------------
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# File Test (Validation)
TEST_DATA_PATH = "/content/drive/MyDrive/KhoaLuan/data/test_grouped_CLEAN_OPTIMIZED.json"

# Danh sách Model cần so sánh
MODELS_TO_EVAL = [
    {
        "name": "ViT5",
        "path": "/content/drive/MyDrive/KhoaLuan/model_final/5ep/model_vit5_final_5ep"
    },
    {
        "name": "BARTpho",
        "path": "/content/drive/MyDrive/KhoaLuan/model_final/5ep/model_bartpho_final_5ep"
    },
    {
        "name": "T5-base",
        "path": "/content/drive/MyDrive/KhoaLuan/model_final/5ep/model_t5_base_final_5ep"
    }
]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 3. HÀM TÍNH ĐIỂM
# ------------------------------------------------------------------------------
metric_bleu = evaluate.load("bleu")
metric_meteor = evaluate.load("meteor")
metric_rouge = evaluate.load("rouge")

def calculate_metrics(predictions, references):
    # BLEU
    results_bleu = metric_bleu.compute(predictions=predictions, references=references)
    # METEOR
    results_meteor = metric_meteor.compute(predictions=predictions, references=references)
    # ROUGE
    results_rouge = metric_rouge.compute(predictions=predictions, references=references)

    return {
        "BLEU": round(results_bleu['bleu'] * 100, 2),
        "METEOR": round(results_meteor['meteor'] * 100, 2),
        "ROUGE-L": round(results_rouge['rougeL'] * 100, 2),
    }

# 4. HÀM SINH CÂU HỎI (QUẢN LÝ RAM CHẶT CHẼ)
# ------------------------------------------------------------------------------
def evaluate_model(model_info, test_data):
    model_name = model_info['name']
    model_path = model_info['path']

    print(f"\n" + "="*50)
    print(f"🔄 Đang khởi động: {model_name}...")
    print(f"📂 Load từ: {model_path}")

    # Kiểm tra đường dẫn
    if not os.path.exists(model_path):
        print(f"❌ Không tìm thấy folder model tại {model_path}. Bỏ qua...")
        return None, None

    # Load Model & Tokenizer
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(DEVICE)
    except Exception as e:
        print(f"❌ Lỗi load model: {e}")
        return None, None

    predictions = []
    print(f"🚀 Đang sinh câu hỏi (Beam Search=5)...")

    # Chạy loop từng mẫu (an toàn nhất cho RAM)
    for item in tqdm(test_data):
        input_text = item['input_text'] # Lấy đúng trường input_text trong JSON

        inputs = tokenizer(input_text, return_tensors="pt", max_length=320, truncation=True).to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_length=256,
                num_beams=5,             # 🔥 CÔNG BẰNG: Tất cả đều dùng Beam 5
                early_stopping=True
            )

        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(pred)

    # 🧹 DỌN DẸP RAM NGAY LẬP TỨC
    print(f"🧹 Đang giải phóng RAM sau khi chạy {model_name}...")
    del model
    del tokenizer
    gc.collect()
    torch.cuda.empty_cache()

    return predictions

# 5. CHẠY THỰC NGHIỆM
# ------------------------------------------------------------------------------
print(f"📂 Đang đọc dữ liệu Test từ {TEST_DATA_PATH}...")
with open(TEST_DATA_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

# ⚠️ LƯU Ý: Test trên 200 mẫu đầu tiên cho nhanh.
# Muốn test hết 100% thì xóa dòng [:200] đi (nhưng sẽ lâu đấy!)
test_data = data[:200]
print(f"👉 Số lượng mẫu đánh giá: {len(test_data)}")

references = [item['output_text'] for item in test_data]
results_list = []
example_outputs = {}

for model_info in MODELS_TO_EVAL:
    preds = evaluate_model(model_info, test_data)

    if preds:
        scores = calculate_metrics(preds, references)
        scores['Model'] = model_info['name']
        results_list.append(scores)

        # Lưu lại 1 ví dụ để so sánh bằng mắt
        example_outputs[model_info['name']] = preds[0]

# 6. HIỂN THỊ KẾT QUẢ & VẼ BIỂU ĐỒ
# ------------------------------------------------------------------------------
if results_list:
    print("\n" + "="*60)
    print("📊 BẢNG TỔNG HỢP KẾT QUẢ SO SÁNH (5 EPOCHS)")
    print("="*60)

    df = pd.DataFrame(results_list)
    # Sắp xếp cột cho đẹp
    df = df[['Model', 'BLEU', 'METEOR', 'ROUGE-L']]
    print(df.to_markdown(index=False))

    # --- VẼ BIỂU ĐỒ ---
    print("\n🎨 Đang vẽ biểu đồ...")
    df.set_index('Model', inplace=True)

    # Vẽ biểu đồ cột nhóm
    ax = df.plot(kind='bar', figsize=(10, 6), rot=0, colormap='viridis')

    plt.title('So sánh hiệu năng các mô hình (Sau 5 Epochs)', fontsize=14, fontweight='bold')
    plt.ylabel('Điểm số (%)')
    plt.xlabel('Mô hình')
    plt.legend(loc='lower right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Hiển thị số trên cột
    for container in ax.containers:
        ax.bar_label(container, fmt='%.1f', padding=3)

    plt.tight_layout()
    plt.show()

    # --- SO SÁNH THỰC TẾ (QUALITATIVE) ---
    print("\n" + "="*60)
    print("📝 SO SÁNH CÂU SINH RA (VÍ DỤ MẪU ĐẦU TIÊN)")
    print("="*60)
    print(f"INPUT: {test_data[0]['input_text']}")
    print(f"GROUND TRUTH: {test_data[0]['output_text']}")
    print("-" * 30)
    for model_name, pred_text in example_outputs.items():
        print(f"🤖 {model_name}: {pred_text}")
        print("-" * 30)

else:
    print("❌ Không có kết quả nào để hiển thị. Hãy kiểm tra lại đường dẫn model.")

#3. Train vit5

In [ ]:
print("⏳ Đang cài đặt thư viện...")
!pip install -q transformers datasets sentencepiece accelerate
print("✅ Cài đặt hoàn tất!")

import os
import shutil
import gc
import torch
from google.colab import drive
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)

# 2. CẤU HÌNH HỆ THỐNG & ĐƯỜNG DẪN
DATA_DIR = "/content/drive/MyDrive/KhoaLuan/data"
TRAIN_FILE = os.path.join(DATA_DIR, "train_grouped_CLEAN_OPTIMIZED.json")
VAL_FILE   = os.path.join(DATA_DIR, "val_grouped_CLEAN_OPTIMIZED.json")
#ten
FINAL_DRIVE_DIR = "/content/drive/MyDrive/KhoaLuan/model_vit5_final_ultra"
TEMP_DIR = "/content/vit5_temp"

MODEL_CHECKPOINT = "vietai/vit5-base"

#thông số
MAX_INPUT_LENGTH = 320
MAX_TARGET_LENGTH = 256

#clean
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
os.makedirs(TEMP_DIR, exist_ok=True)

print("-" * 60)
print(f"✅ Data Train: {TRAIN_FILE}")
print(f"✅ Model Output: {FINAL_DRIVE_DIR}")
print("-" * 60)

# 3. CHUẨN BỊ DỮ LIỆU
print("\n🔄 Đang xử lý dữ liệu...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=MAX_INPUT_LENGTH,
        padding="max_length",
        truncation=True
    )

    labels = tokenizer(
        text_target=examples["output_text"],
        max_length=MAX_TARGET_LENGTH,
        padding="max_length",
        truncation=True
    )
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Load và Map dữ liệu
try:
    dataset = load_dataset("json", data_files={"train": TRAIN_FILE, "validation": VAL_FILE})
except Exception as e:
    print(f"❌ Lỗi load data: {e}")
    raise e

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
print(f"✅ Đã xử lý xong {len(tokenized_datasets['train'])} mẫu train.")

# 4. CẤU HÌNH TRAINING

gc.collect()
torch.cuda.empty_cache()

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

training_args = Seq2SeqTrainingArguments(
    output_dir=TEMP_DIR,
    learning_rate=3e-4,
    num_train_epochs=15,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    label_smoothing_factor=0.1,
    weight_decay=0.02,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    fp16=True,

    # cấu hình lưu
    save_total_limit=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    predict_with_generate=False,
    logging_steps=100,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
   #xet
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

# 5. BẮT ĐẦU TRAIN
trainer.train()

# Xóa rác
if os.path.exists(FINAL_DRIVE_DIR):
    shutil.rmtree(FINAL_DRIVE_DIR)

trainer.save_model(FINAL_DRIVE_DIR)
tokenizer.save_pretrained(FINAL_DRIVE_DIR)

print("-" * 60)
print(f"🎉 HOÀN TẤT! Model tối ưu nhất đã được lưu tại:")
print(f"👉 {FINAL_DRIVE_DIR}")
print("-" * 60)

#6. TEST
from transformers import pipeline

print("\n🤖 Đang test thử 1 mẫu...")
# Load lại model vừa lưu để chắc chắn nó hoạt động
generator = pipeline("text2text-generation", model=FINAL_DRIVE_DIR, tokenizer=FINAL_DRIVE_DIR, device=0)

test_context = "Quang hợp là quá trình sử dụng năng lượng ánh sáng để tổng hợp chất hữu cơ từ các nguyên liệu vô cơ."
input_prompt = f"môn: Sinh học | chủ đề: Trao đổi chất | mức độ: Nhận biết | ngữ cảnh: {test_context}"

# Test với tham số sinh chuẩn (Beam 5)
output = generator(
    input_prompt,
    max_length=256,
    num_beams=5,
    length_penalty=1.0
)

print(f"\n📝 INPUT: {input_prompt}")
print(f"✅ OUTPUT: {output[0]['generated_text']}")

#4. Test mô hình viT5

In [6]:
import torch
import re
import os
import random
import pandas as pd
from datetime import datetime
from difflib import SequenceMatcher
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from google.colab import drive

# 1. KẾT NỐI DRIVE & LOAD MODEL
if not os.path.exists('/content/drive'): drive.mount('/content/drive')
MODEL_PATH = "/content/drive/MyDrive/KhoaLuan/model_final/v2/model_vit5_final_ultra"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"⏳ Đang tải model từ: {MODEL_PATH} ...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(DEVICE)
print("✅ Hệ thống đã sẵn sàng!")

# 2. KHO ĐÁP ÁN NHIỄU (10 MÔN HỌC)
BACKUP_OPTIONS = {
    "Toán học": ["Số hữu tỉ", "Hàm số bậc nhất", "Định lí Pythagore", "Tam giác đồng dạng", "Hệ phương trình", "Đạo hàm", "Tích phân"],
    "Vật lý": ["Định luật Newton", "Điện trở", "Phản xạ ánh sáng", "Dao động cơ", "Sóng dừng", "Dòng điện xoay chiều"],
    "Hóa học": ["Bảng tuần hoàn", "Liên kết cộng hóa trị", "Phản ứng thế", "Kim loại kiềm", "Ancol", "Polime", "Sắt và hợp chất"],
    "Sinh học": ["Tế bào nhân thực", "Quang hợp", "Nguyên phân", "Di truyền học", "Hệ sinh thái", "Chọn lọc tự nhiên"],
    "Lịch sử": ["Phong trào Cần vương", "Cách mạng tháng Tám", "Chiến dịch Điện Biên Phủ", "Toàn cầu hóa", "Chiến tranh lạnh"],
    "Địa lý": ["Địa hình nhiệt đới", "Gió mùa", "Mật độ dân số", "Chuyển dịch cơ cấu kinh tế", "Vùng kinh tế trọng điểm"],
    "Tin học": ["Hệ điều hành", "Mạng máy tính", "Trình duyệt Web", "Ngôn ngữ lập trình", "Cơ sở dữ liệu", "Trí tuệ nhân tạo"],
    "Ngữ văn": ["Hiện thực phê phán", "Văn học lãng mạn", "Nhân đạo", "Tình huống truyện", "Bút pháp nghệ thuật", "Tố Hữu", "Nam Cao"],
    "Công nghệ": ["Bản vẽ kỹ thuật", "Gia công cơ khí", "Mạch điện điều khiển", "Động cơ đốt trong", "Nông nghiệp công nghệ cao"],
    "Giáo dục Kinh tế và Pháp luật": ["Thị trường", "Cung cầu", "Lạm phát", "Thuế", "Quyền dân sự", "Hiến pháp", "Pháp luật lao động"]
}

# 3. CÁC HÀM XỬ LÝ LOGIC (HẬU KỲ)
def get_smart_options_safe(options_dict, context, subject):
    seen_values = set()
    final_opts = {}
    backups = BACKUP_OPTIONS.get(subject, ["Đáp án A", "Đáp án B"]).copy()
    random.shuffle(backups)
    for key, value in options_dict.items():
        clean_val = value.lower().replace(".", "").strip()
        if clean_val not in seen_values and len(clean_val) > 2:
            seen_values.add(clean_val)
            final_opts[key] = value
        else:
            if backups:
                new_val = backups.pop()
                while new_val.lower() in seen_values and backups:
                    new_val = backups.pop()
                final_opts[key] = new_val
                seen_values.add(new_val.lower())
    return dict(sorted(final_opts.items()))

def get_best_match(context, options_dict, model_ans_char):
    best_key = model_ans_char
    highest_score = 0
    for key, content in options_dict.items():
        match_ratio = SequenceMatcher(None, content, context).ratio()
        if match_ratio > highest_score:
            highest_score = match_ratio
            best_key = key
    return best_key if highest_score > 0.5 else model_ans_char

def run_test_item(subject, topic, level, context):
    input_text = f"môn: {subject} | chủ đề: {topic} | mức độ: {level} | ngữ cảnh: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids, max_length=320, do_sample=True,
            num_beams=3, temperature=0.7, repetition_penalty=1.5,
            no_repeat_ngram_size=3, early_stopping=True
        )

    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pattern = r"(?si)question\s*[:\-]\s*(.*?)\s*options\s*[:\-]\s*(.*?)\s*answer\s*[:\-]\s*(.*)"
    match = re.search(pattern, raw_output)

    if match:
        q = match.group(1).strip()
        opts_str = match.group(2).strip()
        ans_raw = match.group(3).strip()
        opts_list = re.findall(r"([A-D])\.\s*(.*?)(?=\s*[A-D]\.|$)", opts_str + " ", re.DOTALL)
        options_dict = {k: v.strip() for k, v in opts_list}

        # Xử lý làm sạch và sửa lỗi options
        options_dict = get_smart_options_safe(options_dict, context, subject)
        model_ans_char = ans_raw[0].upper() if len(ans_raw) > 0 else "A"
        final_key = get_best_match(context, options_dict, model_ans_char)

        # In kết quả ra màn hình
        print(f"📌 [{level.upper()}] {q}")
        print(f"   " + "  |  ".join([f"{k}. {v}" for k, v in options_dict.items()]))
        print(f"   ✅ ĐÁP ÁN: {final_key}. {options_dict.get(final_key)}")
    else:
        print(f"❌ Lỗi format: {raw_output[:100]}...")
    print("-" * 50)

# ==============================================================================
# 4. DANH SÁCH DỮ LIỆU CẦN TEST (BẠN TỰ NHẬP VÀO ĐÂY)
# ==============================================================================
# ==============================================================================
# 🚀 FULL DATA TEST: 4 MÔN x 3 MỨC ĐỘ = 12 MẪU KIỂM THỬ
# ==============================================================================
my_test_list = [
    # --- 1. ĐỊA LÝ (Chuyển dịch cơ cấu kinh tế) ---
    {
        "subject": "Địa lý", "topic": "Chuyển dịch cơ cấu kinh tế ĐBSH", "level": "Nhận biết",
        "context": "Đồng bằng sông Hồng là vùng có vị trí chiến lược quan trọng, đóng góp lớn vào GDP cả nước. Hiện nay, cơ cấu kinh tế của vùng đang có sự chuyển dịch mạnh mẽ theo hướng công nghiệp hóa, hiện đại hóa. Tỷ trọng ngành nông, lâm nghiệp và thủy sản có xu hướng giảm, trong khi tỷ trọng ngành công nghiệp, xây dựng và dịch vụ tăng lên nhanh chóng. Tuy nhiên, sự chuyển dịch này vẫn còn chậm so với tiềm năng thực tế. Nguyên nhân chủ yếu là do kết cấu hạ tầng chưa đồng bộ, chất lượng nguồn lao động chưa đáp ứng được yêu cầu của các ngành công nghệ cao. Bên cạnh đó, sức ép về mật độ dân số quá cao (lớn nhất cả nước) cũng gây ra nhiều khó khăn trong việc giải quyết việc làm và bảo vệ tài nguyên môi trường bền vững cho toàn vùng."
    },
    {
        "subject": "Địa lý", "topic": "Chuyển dịch cơ cấu kinh tế ĐBSH", "level": "Thông hiểu",
        "context": "Đồng bằng sông Hồng là vùng có vị trí chiến lược quan trọng, đóng góp lớn vào GDP cả nước. Hiện nay, cơ cấu kinh tế của vùng đang có sự chuyển dịch mạnh mẽ theo hướng công nghiệp hóa, hiện đại hóa. Tỷ trọng ngành nông, lâm nghiệp và thủy sản có xu hướng giảm, trong khi tỷ trọng ngành công nghiệp, xây dựng và dịch vụ tăng lên nhanh chóng. Tuy nhiên, sự chuyển dịch này vẫn còn chậm so với tiềm năng thực tế. Nguyên nhân chủ yếu là do kết cấu hạ tầng chưa đồng bộ, chất lượng nguồn lao động chưa đáp ứng được yêu cầu của các ngành công nghệ cao. Bên cạnh đó, sức ép về mật độ dân số quá cao (lớn nhất cả nước) cũng gây ra nhiều khó khăn trong việc giải quyết việc làm và bảo vệ tài nguyên môi trường bền vững cho toàn vùng."
    },
    {
        "subject": "Địa lý", "topic": "Chuyển dịch cơ cấu kinh tế ĐBSH", "level": "Vận dụng",
        "context": "Đồng bằng sông Hồng là vùng có vị trí chiến lược quan trọng, đóng góp lớn vào GDP cả nước. Hiện nay, cơ cấu kinh tế của vùng đang có sự chuyển dịch mạnh mẽ theo hướng công nghiệp hóa, hiện đại hóa. Tỷ trọng ngành nông, lâm nghiệp và thủy sản có xu hướng giảm, trong khi tỷ trọng ngành công nghiệp, xây dựng và dịch vụ tăng lên nhanh chóng. Tuy nhiên, sự chuyển dịch này vẫn còn chậm so với tiềm năng thực tế. Nguyên nhân chủ yếu là do kết cấu hạ tầng chưa đồng bộ, chất lượng nguồn lao động chưa đáp ứng được yêu cầu của các ngành công nghệ cao. Bên cạnh đó, sức ép về mật độ dân số quá cao (lớn nhất cả nước) cũng gây ra nhiều khó khăn trong việc giải quyết việc làm và bảo vệ tài nguyên môi trường bền vững cho toàn vùng."
    },

    # --- 2. SINH HỌC (Cấu tạo và chức năng của Ti thể) ---
    {
        "subject": "Sinh học", "topic": "Ti thể trong tế bào", "level": "Nhận biết",
        "context": "Ti thể là một trong những bào quan quan trọng nhất trong tế bào nhân thực, được bao bọc bởi hai lớp màng sinh chất. Màng ngoài trơn nhẵn, trong khi màng trong gấp nếp tạo thành các mào, nơi chứa các enzyme hô hấp. Bên trong ti thể là chất nền chứa DNA và ribosome riêng, cho phép chúng tự nhân đôi độc lập với sự phân chia của tế bào. Chức năng chính của ti thể là nơi diễn ra quá trình hô hấp tế bào, chuyển hóa năng lượng từ các hợp chất hữu cơ thành ATP – nguồn năng lượng trực tiếp cho mọi hoạt động sống. Số lượng ti thể trong mỗi loại tế bào là không giống nhau, phụ thuộc vào nhu cầu năng lượng của loại tế bào đó. Ví dụ, tế bào cơ tim hoặc tế bào gan thường chứa hàng nghìn ti thể để đảm bảo duy trì công năng hoạt động liên tục."
    },
    {
        "subject": "Sinh học", "topic": "Ti thể trong tế bào", "level": "Thông hiểu",
        "context": "Ti thể là một trong những bào quan quan trọng nhất trong tế bào nhân thực, được bao bọc bởi hai lớp màng sinh chất. Màng ngoài trơn nhẵn, trong khi màng trong gấp nếp tạo thành các mào, nơi chứa các enzyme hô hấp. Bên trong ti thể là chất nền chứa DNA và ribosome riêng, cho phép chúng tự nhân đôi độc lập với sự phân chia của tế bào. Chức năng chính của ti thể là nơi diễn ra quá trình hô hấp tế bào, chuyển hóa năng lượng từ các hợp chất hữu cơ thành ATP – nguồn năng lượng trực tiếp cho mọi hoạt động sống. Số lượng ti thể trong mỗi loại tế bào là không giống nhau, phụ thuộc vào nhu cầu năng lượng của loại tế bào đó. Ví dụ, tế bào cơ tim hoặc tế bào gan thường chứa hàng nghìn ti thể để đảm bảo duy trì công năng hoạt động liên tục."
    },
    {
        "subject": "Sinh học", "topic": "Ti thể trong tế bào", "level": "Vận dụng",
        "context": "Ti thể là một trong những bào quan quan trọng nhất trong tế bào nhân thực, được bao bọc bởi hai lớp màng sinh chất. Màng ngoài trơn nhẵn, trong khi màng trong gấp nếp tạo thành các mào, nơi chứa các enzyme hô hấp. Bên trong ti thể là chất nền chứa DNA và ribosome riêng, cho phép chúng tự nhân đôi độc lập với sự phân chia của tế bào. Chức năng chính của ti thể là nơi diễn ra quá trình hô hấp tế bào, chuyển hóa năng lượng từ các hợp chất hữu cơ thành ATP – nguồn năng lượng trực tiếp cho mọi hoạt động sống. Số lượng ti thể trong mỗi loại tế bào là không giống nhau, phụ thuộc vào nhu cầu năng lượng của loại tế bào đó. Ví dụ, tế bào cơ tim hoặc tế bào gan thường chứa hàng nghìn ti thể để đảm bảo duy trì công năng hoạt động liên tục."
    },

    # --- 3. TIN HỌC (Giao thức HTTPS) ---
    {
        "subject": "Tin học", "topic": "Giao thức HTTPS", "level": "Nhận biết",
        "context": "Trong kỷ nguyên số, an toàn thông tin là ưu tiên hàng đầu khi thực hiện các giao dịch trên mạng Internet. Giao thức HTTPS (Hypertext Transfer Protocol Secure) đã ra đời như một phiên bản bảo mật của HTTP truyền thống. HTTPS hoạt động bằng cách kết hợp giao thức HTTP với chứng chỉ SSL/TLS để mã hóa dữ liệu truyền đi giữa trình duyệt của người dùng và máy chủ web. Quá trình này đảm bảo rằng các thông tin nhạy cảm như mật khẩu, tài khoản ngân hàng hoặc dữ liệu cá nhân không bị kẻ xấu đánh cắp hoặc thay đổi trên đường truyền. Một website sử dụng HTTPS thường có biểu tượng ổ khóa xanh trên thanh địa chỉ, giúp tăng độ tin cậy cho người dùng."
    },
    {
        "subject": "Tin học", "topic": "Giao thức HTTPS", "level": "Thông hiểu",
        "context": "Trong kỷ nguyên số, an toàn thông tin là ưu tiên hàng đầu khi thực hiện các giao dịch trên mạng Internet. Giao thức HTTPS (Hypertext Transfer Protocol Secure) đã ra đời như một phiên bản bảo mật của HTTP truyền thống. HTTPS hoạt động bằng cách kết hợp giao thức HTTP với chứng chỉ SSL/TLS để mã hóa dữ liệu truyền đi giữa trình duyệt của người dùng và máy chủ web. Quá trình này đảm bảo rằng các thông tin nhạy cảm như mật khẩu, tài khoản ngân hàng hoặc dữ liệu cá nhân không bị kẻ xấu đánh cắp hoặc thay đổi trên đường truyền. Một website sử dụng HTTPS thường có biểu tượng ổ khóa xanh trên thanh địa chỉ, giúp tăng độ tin cậy cho người dùng."
    },
    {
        "subject": "Tin học", "topic": "Giao thức HTTPS", "level": "Vận dụng",
        "context": "Trong kỷ nguyên số, an toàn thông tin là ưu tiên hàng đầu khi thực hiện các giao dịch trên mạng Internet. Giao thức HTTPS (Hypertext Transfer Protocol Secure) đã ra đời như một phiên bản bảo mật của HTTP truyền thống. HTTPS hoạt động bằng cách kết hợp giao thức HTTP with SSL/TLS certificate to encrypt data transmitted between browser and web server. Quá trình này đảm bảo rằng các thông tin nhạy cảm như mật khẩu, tài khoản ngân hàng hoặc dữ liệu cá nhân không bị kẻ xấu đánh cắp hoặc thay đổi trên đường truyền. Việc triển khai HTTPS không chỉ bảo vệ quyền riêng tư mà còn là một tiêu chuẩn quan trọng để các công cụ tìm kiếm xếp hạng uy tín cho trang web."
    },

    # --- 4. NGỮ VĂN (Vợ nhặt) ---
    {
        "subject": "Ngữ văn", "topic": "Giá trị nhân đạo trong Vợ nhặt", "level": "Nhận biết",
        "context": "Truyện ngắn 'Vợ nhặt' của nhà văn Kim Lân không chỉ khắc họa bức tranh thê thảm của nạn đói năm 1945 mà còn tỏa sáng vẻ đẹp của tình người và khát vọng sống mãnh liệt. Trong hoàn cảnh cái chết bủa vây, nhân vật Tràng vẫn liều lĩnh đưa một người đàn bà xa lạ về làm vợ. Hành động này không phải là sự bồng bột của bản năng mà xuất phát từ lòng nhân hậu và niềm tin vào tương lai. Sự hiện diện của nàng dâu mới đã làm thay đổi hoàn toàn không khí u ám của xóm ngụ cư và đem lại sức sống cho bà cụ Tứ. Qua đó, tác giả khẳng định một chân lý nghệ thuật sâu sắc: dù ở bên bờ vực của cái chết, con người vẫn không ngừng khát khao hạnh phúc, vẫn đùm bọc và yêu thương nhau để vượt qua nghịch cảnh. Đó chính là giá trị nhân đạo cao cả."
    },
    {
        "subject": "Ngữ văn", "topic": "Giá trị nhân đạo trong Vợ nhặt", "level": "Thông hiểu",
        "context": "Truyện ngắn 'Vợ nhặt' của nhà văn Kim Lân không chỉ khắc họa bức tranh thê thảm của nạn đói năm 1945 mà còn tỏa sáng vẻ đẹp của tình người và khát vọng sống mãnh liệt. Trong hoàn cảnh cái chết bủa vây, nhân vật Tràng vẫn liều lĩnh đưa một người đàn bà xa lạ về làm vợ. Hành động này không phải là sự bồng bột của bản năng mà xuất phát từ lòng nhân hậu và niềm tin vào tương lai. Sự hiện diện của nàng dâu mới đã làm thay đổi hoàn toàn không khí u ám của xóm ngụ cư và đem lại sức sống cho bà cụ Tứ. Qua đó, tác giả khẳng định một chân lý nghệ thuật sâu sắc: dù ở bên bờ vực của cái chết, con người vẫn không ngừng khát khao hạnh phúc, vẫn đùm bọc và yêu thương nhau để vượt qua nghịch cảnh. Đó chính là giá trị nhân đạo cao cả."
    },
    {
        "subject": "Ngữ văn", "topic": "Giá trị nhân đạo trong Vợ nhặt", "level": "Vận dụng",
        "context": "Truyện ngắn 'Vợ nhặt' của nhà văn Kim Lân không chỉ khắc họa bức tranh thê thảm của nạn đói năm 1945 mà còn tỏa sáng vẻ đẹp của tình người và khát vọng sống mãnh liệt. Trong hoàn cảnh cái chết bủa vây, nhân vật Tràng vẫn liều lĩnh đưa một người đàn bà xa lạ về làm vợ. Hành động này không phải là sự bồng bột của bản năng mà xuất phát từ lòng nhân hậu và niềm tin vào tương lai. Sự hiện diện của nàng dâu mới đã làm thay đổi hoàn toàn không khí u ám của xóm ngụ cư và đem lại sức sống cho bà cụ Tứ. Qua đó, tác giả khẳng định một chân lý nghệ thuật sâu sắc: dù ở bên bờ vực của cái chết, con người vẫn không ngừng khát khao hạnh phúc, vẫn đùm bọc và yêu thương nhau để vượt qua nghịch cảnh. Đó chính là giá trị nhân đạo cao cả."
    }
]

# CHẠY LOOP QUA DANH SÁCH 12 CÂU HỎI
for item in my_test_list:
    run_test_item(item['subject'], item['topic'], item['level'], item['context'])

⏳ Đang tải model từ: /content/drive/MyDrive/KhoaLuan/model_final/v2/model_vit5_final_ultra ...
✅ Hệ thống đã sẵn sàng!
📌 [NHẬN BIẾT] Vùng nào có vị trí chiến lược quan trọng, đóng góp lớn vào GDP cả nước?
   A. Tây Nguyên.  |  B. Đồng bằng sông Hồng.  |  C. Trung du và miền núi Bắc Bộ.  |  D. Duyên hải Nam Trung Bộ.
   ✅ ĐÁP ÁN: B. Đồng bằng sông Hồng.
--------------------------------------------------
📌 [THÔNG HIỂU] Nguyên nhân chủ yếu khiến tỉ trọng ngành nông, lâm nghiệp và thủy sản ở ĐBSH còn chậm là gì?
   A. Thiếu nguồn lao động  |  B. Thiếu tài nguyên rừng  |  C. Thiếu vốn đầu tư  |  D. Kết cấu hạ tầng chưa đồng bộ, chất lượng nguồn lao Động chưa đáp ứng được yêu cầu của các ngành công nghệ cao
   ✅ ĐÁP ÁN: D. Kết cấu hạ tầng chưa đồng bộ, chất lượng nguồn lao Động chưa đáp ứng được yêu cầu của các ngành công nghệ cao
--------------------------------------------------
📌 [VẬN DỤNG] Nguyên nhân chủ yếu gây ra sự chuyển dịch cơ cấu kinh tế chậm ở Đồng bằng sông Hồng là gì?
   A. Th

#5. Demo

In [13]:
# 1. CÀI ĐẶT THƯ VIỆN
!pip install -q gradio transformers torch pandas openpyxl

import gradio as gr
import torch
import re
import os
import random
import pandas as pd
from datetime import datetime
from difflib import SequenceMatcher
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from google.colab import drive

# 2. LOAD MODEL
if not os.path.exists('/content/drive'): drive.mount('/content/drive')
MODEL_PATH = "/content/drive/MyDrive/KhoaLuan/model_final/v2/model_vit5_final_ultra"
# Đường dẫn file lưu trữ trên Drive
SAVE_PATH = "/content/drive/MyDrive/KhoaLuan/generated_quizzes.xlsx"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"⏳ Đang tải model từ: {MODEL_PATH} ...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH).to(DEVICE)
print("✅ Hệ thống đã sẵn sàng!")

# 3. KHO ĐÁP ÁN NHIỄU (Giữ nguyên từ code của bạn)
BACKUP_OPTIONS = {
    "Toán học": ["Số hữu tỉ", "Hàm số bậc nhất", "Định lí Pythagore", "Tam giác đồng dạng", "Hệ phương trình", "Đạo hàm", "Tích phân"],
    "Vật lý": ["Định luật Newton", "Điện trở", "Phản xạ ánh sáng", "Dao động cơ", "Sóng dừng", "Dòng điện xoay chiều"],
    "Hóa học": ["Bảng tuần hoàn", "Liên kết cộng hóa trị", "Phản ứng thế", "Kim loại kiềm", "Ancol", "Polime", "Sắt và hợp chất"],
    "Sinh học": ["Tế bào nhân thực", "Quang hợp", "Nguyên phân", "Di truyền học", "Hệ sinh thái", "Chọn lọc tự nhiên"],
    "Lịch sử": ["Phong trào Cần vương", "Cách mạng tháng Tám", "Chiến dịch Điện Biên Phủ", "Toàn cầu hóa", "Chiến tranh lạnh"],
    "Địa lý": ["Địa hình nhiệt đới", "Gió mùa", "Mật độ dân số", "Chuyển dịch cơ cấu kinh tế", "Vùng kinh tế trọng điểm"],
    "Tin học": ["Hệ điều hành", "Mạng máy tính", "Trình duyệt Web", "Ngôn ngữ lập trình", "Cơ sở dữ liệu", "Trí tuệ nhân tạo"],
    "Ngữ văn": ["Hiện thực phê phán", "Văn học lãng mạn", "Nhân đạo", "Tình huống truyện", "Bút pháp nghệ thuật", "Tố Hữu", "Nam Cao"],
    "Công nghệ": ["Bản vẽ kỹ thuật", "Gia công cơ khí", "Mạch điện điều khiển", "Động cơ đốt trong", "Nông nghiệp công nghệ cao"],
    "Giáo dục Kinh tế và Pháp luật": ["Thị trường", "Cung cầu", "Lạm phát", "Thuế", "Quyền dân sự", "Hiến pháp", "Pháp luật lao động"]
}

# --- CÁC HÀM XỬ LÝ LOGIC ---
def get_smart_options_safe(options_dict, context, subject):
    seen_values = set()
    final_opts = {}
    backups = BACKUP_OPTIONS.get(subject, ["Phương án A", "Phương án B"]).copy()
    random.shuffle(backups)
    for key, value in options_dict.items():
        clean_val = value.lower().replace(".", "").strip()
        if clean_val not in seen_values and len(clean_val) > 2:
            seen_values.add(clean_val)
            final_opts[key] = value
        else:
            if backups:
                new_val = backups.pop()
                while new_val.lower() in seen_values and backups:
                    new_val = backups.pop()
                final_opts[key] = new_val
                seen_values.add(new_val.lower())
    return dict(sorted(final_opts.items()))

def get_best_match(context, options_dict, model_ans_char):
    best_key = model_ans_char
    highest_score = 0
    for key, content in options_dict.items():
        match_ratio = SequenceMatcher(None, content, context).ratio()
        if match_ratio > highest_score:
            highest_score = match_ratio
            best_key = key
    return best_key if highest_score > 0.5 else model_ans_char

def generate_quiz_gradio(subject, topic, level, context):
    input_text = f"môn: {subject} | chủ đề: {topic} | mức độ: {level} | ngữ cảnh: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(DEVICE)
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids, max_length=320, do_sample=True,
            num_beams=3, temperature=0.7, repetition_penalty=1.5,
            no_repeat_ngram_size=3, early_stopping=True
        )
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pattern = r"(?si)question\s*[:\-]\s*(.*?)\s*options\s*[:\-]\s*(.*?)\s*answer\s*[:\-]\s*(.*)"
    match = re.search(pattern, raw_output)
    if match:
        q = match.group(1).strip()
        opts_str = match.group(2).strip()
        ans_raw = match.group(3).strip()
        opts_list = re.findall(r"([A-D])\.\s*(.*?)(?=\s*[A-D]\.|$)", opts_str + " ", re.DOTALL)
        options_dict = {k: v.strip() for k, v in opts_list}
        options_dict = get_smart_options_safe(options_dict, context, subject)
        model_ans_char = ans_raw[0].upper() if len(ans_raw) > 0 else "A"
        final_key = get_best_match(context, options_dict, model_ans_char)
        full_opts = "\n".join([f"{k}. {v}" for k, v in options_dict.items()])
        correct_ans = f"{final_key}. {options_dict.get(final_key)}"
        return q, full_opts, correct_ans
    return "⚠️ Lỗi format output", "", ""

# --- HÀM LƯU DỮ LIỆU ---
def save_to_excel(subject, topic, level, context, question, options, answer):
    if not question or "⚠️" in question:
        return "❌ Không có câu hỏi hợp lệ để lưu!"

    new_data = {
        "Thời gian": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")],
        "Môn học": [subject],
        "Chủ đề": [topic],
        "Mức độ": [level],
        "Ngữ cảnh": [context],
        "Câu hỏi": [question],
        "Lựa chọn": [options.replace("\n", " | ")],
        "Đáp án đúng": [answer]
    }
    df_new = pd.DataFrame(new_data)

    try:
        if os.path.exists(SAVE_PATH):
            df_old = pd.read_excel(SAVE_PATH)
            df_final = pd.concat([df_old, df_new], ignore_index=True)
        else:
            df_final = df_new

        df_final.to_excel(SAVE_PATH, index=False)
        return f"✅ Đã lưu vào Drive: {SAVE_PATH}"
    except Exception as e:
        return f"❌ Lỗi khi lưu: {str(e)}"


# 5. GIAO DIỆN GRADIO (CÂN BẰNG KÍCH THƯỚC CÁC Ô ĐẦU RA)
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎓 HỆ THỐNG SINH CÂU HỎI TRẮC NGHIỆM AI (ViT5)")

    with gr.Row():
        with gr.Column():
            subject_in = gr.Dropdown(list(BACKUP_OPTIONS.keys()), label="📚 Môn học", value="Ngữ văn")
            topic_in = gr.Textbox(label="🔍 Chủ đề", placeholder="Ví dụ: Quy luật cung cầu...")
            level_in = gr.Radio(["Nhận biết", "Thông hiểu", "Vận dụng"], label="🧠 Mức độ tư duy", value="Nhận biết")
            context_in = gr.Textbox(label="📝 Ngữ cảnh", lines=5, placeholder="Nhập đoạn văn bản để AI phân tích...")
            btn_gen = gr.Button("🔥 SINH CÂU HỎI", variant="primary")

        with gr.Column():
            # Ô Câu hỏi: Tự động co giãn tối đa 10 dòng
            res_q = gr.Textbox(
                label="❓ Câu hỏi sinh ra",
                lines=3,
                max_lines=10,
                interactive=False
            )
            # Ô Phương án: Tự động co giãn tối đa 15 dòng
            res_opts = gr.Textbox(
                label="📋 Các phương án",
                lines=5,
                max_lines=15,
                interactive=False
            )
            # Ô Đáp án đúng: Đã chỉnh lines=3 và max_lines=10 để bằng với ô Câu hỏi
            res_ans = gr.Textbox(
                label="✅ Đáp án đúng",
                lines=3,           # Thiết lập mặc định 3 dòng giống res_q
                max_lines=10,      # Tự động giãn nở tối đa 10 dòng nương theo nội dung
                interactive=False
            )
            btn_save = gr.Button("💾 LƯU VÀO DRIVE", variant="secondary")
            status_msg = gr.Markdown()

    # Thiết lập các sự kiện Click (giữ nguyên logic xử lý)
    btn_gen.click(
        fn=generate_quiz_gradio,
        inputs=[subject_in, topic_in, level_in, context_in],
        outputs=[res_q, res_opts, res_ans]
    )

    btn_save.click(
        fn=save_to_excel,
        inputs=[subject_in, topic_in, level_in, context_in, res_q, res_opts, res_ans],
        outputs=[status_msg]
    )

    gr.Examples(
        examples=[
            ["Giáo dục Kinh tế và Pháp luật", "Cung - Cầu", "Thông hiểu", "Khi giá cả hàng hóa tăng lên, các doanh nghiệp mở rộng sản xuất làm cung tăng."],
            ["Toán học", "Hàm số", "Nhận biết", "Hàm số y = ax + b đồng biến khi a > 0."],
        ],
        inputs=[subject_in, topic_in, level_in, context_in]
    )

demo.launch(share=True, debug=True)

⏳ Đang tải model từ: /content/drive/MyDrive/KhoaLuan/model_final/v2/model_vit5_final_ultra ...
✅ Hệ thống đã sẵn sàng!


/tmp/ipython-input-2636561650.py:128: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://52b059033845ff2de1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://52b059033845ff2de1.gradio.live
